In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, window
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

In [2]:
spark = SparkSession.builder.appName("spark windowing app").getOrCreate()

24/02/17 17:14:52 WARN Utils: Your hostname, Abhishek-ROG resolves to a loopback address: 127.0.1.1; using 172.20.83.248 instead (on interface eth0)
24/02/17 17:14:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/17 17:14:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/17 17:14:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/17 17:14:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
data = spark.readStream.format("socket").option("host","localhost").option("port",9999).load()

24/02/17 17:14:56 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
structured_data_df = data.selectExpr("split(value, ',') as data") \
    .selectExpr(
        "CAST(data[0] AS TIMESTAMP) as Timestamp",
        "CAST(data[1] AS INT) as ServerNumber",
        "CAST(data[2] AS STRING) as ServerType",
        "CAST(data[3] AS STRING) as Location",
        "CAST(data[4] AS STRING) as IPAddress",
        "CAST(data[5] AS STRING) as AlertState",
        "CAST(data[6] AS STRING) as AlertDescription"
    )

filtered_alerts = structured_data_df.na.drop()

In [5]:
grouped_alerts = filtered_alerts.withWatermark('Timestamp','5 minutes').groupBy(window('Timestamp', '5 minutes', '5 minutes'), 'ServerType', 'Location', 'AlertState').count()

In [6]:
query = grouped_alerts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

24/02/17 17:14:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-87c4f002-a7ea-43ee-88f6-d1c68377b683. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/17 17:14:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+--------+----------+-----+
|window|ServerType|Location|AlertState|count|
+------+----------+--------+----------+-----+
+------+----------+--------+----------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
+--------------------+------------------+--------+----------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15...|   Database Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|   Database Server|      US|      Info|    2|
|{2024-02-17 17:15...|        Web Server|      Uk|      Info|    1|
|{2024-02-17 17:15...|        Web Server|   India|  Critical|    1|
+--------------------+------------------+--------+----------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|   India|      Info|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15...|   Database Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|   Database Server|      US|      Info|    2|
|{2024-02-17 17:15...|        Web Server|      Uk|      Info|    1|
|{2024-02-17 17:15...|        Web Server|   India|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|      US|  Critical|    2|
+--------------------+------------------+--------+----------+-----+



...

KeyboardInterrupt: 

-------------------------------------------
Batch: 11
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:15...|        Web Server|      Uk|   Warning|    2|
|{2024-02-17 17:15...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|   Database Server|      US|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      US|      Info|    2|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|   India|      Info|    1|
|{2024-02-17 17:15...|   Database Server|      Uk|   Warning|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15

-------------------------------------------
Batch: 12
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:15...|        Web Server|      Uk|   Warning|    3|
|{2024-02-17 17:15...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|   Database Server|      US|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      US|      Info|    2|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|   India|      Info|    1|
|{2024-02-17 17:15...|   Database Server|      Uk|   Warning|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15

-------------------------------------------
Batch: 13
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:15...|        Web Server|      Uk|   Warning|    3|
|{2024-02-17 17:15...|        Web Server|      Uk|  Critical|    2|
|{2024-02-17 17:15...|   Database Server|      US|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      US|      Info|    3|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|   India|      Info|    1|
|{2024-02-17 17:15...|   Database Server|      Uk|   Warning|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15

-------------------------------------------
Batch: 14
-------------------------------------------
+--------------------+------------------+--------+----------+-----+
|              window|        ServerType|Location|AlertState|count|
+--------------------+------------------+--------+----------+-----+
|{2024-02-17 17:10...|   Database Server|   India|      Info|    1|
|{2024-02-17 17:15...|        Web Server|      Uk|   Warning|    3|
|{2024-02-17 17:15...|        Web Server|      Uk|  Critical|    2|
|{2024-02-17 17:15...|   Database Server|      US|  Critical|    2|
|{2024-02-17 17:15...|Application Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|Application Server|      US|      Info|    3|
|{2024-02-17 17:10...|        Web Server|      Uk|  Critical|    1|
|{2024-02-17 17:15...|        Web Server|   India|      Info|    2|
|{2024-02-17 17:15...|   Database Server|      Uk|   Warning|    1|
|{2024-02-17 17:10...|Application Server|   India|      Info|    1|
|{2024-02-17 17:15